<a href="https://colab.research.google.com/github/jay19patel/-Django_E_Com_full_Website/blob/main/Stock_Data_fetch_from_NSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from requests import Session
from urllib.parse import urljoin
from functools import wraps
import pandas as pd


In [111]:
class MyNSE:
    def __init__(self):
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
            "Referer": "https://www.nseindia.com/get-quotes/equity?symbol=SBIN",
            "X-Requested-With": "XMLHttpRequest",
        }
        self.base_url = "https://www.nseindia.com"
        self.s = Session()
        self.s.headers.update(self.headers)
        self.ssl_verify = True
        self._initialize_session()

    def _initialize_session(self):
        if "nseappid" not in self.s.cookies:
            url = urljoin(self.base_url, "/get-quotes/equity")
            self.s.get(url, verify=self.ssl_verify)
            print("Session Generated Successfully")

    def ensure_session(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            if "nseappid" not in self.s.cookies:
                url = urljoin(self.base_url, "/get-quotes/equity")
                self.s.get(url, verify=self.ssl_verify)
                print("Session Generated Successfully")
            return func(self, *args, **kwargs)
        return wrapper

    # NSE Stock List
    @ensure_session
    def getNSEStockList(self, indexName):
        url = f"https://www.nseindia.com/api/equity-stockIndices?index={indexName}"
        response = self.s.get(url, verify=self.ssl_verify)

        if response.status_code == 200:
            return pd.DataFrame(response.json()['data'][1:])
        else:
            raise Exception(f"Failed to fetch data: {response.status_code} {response.text}")

    # Financial Results (All Values are in ₹ Lakhs.)
    @ensure_session
    def getFinancialResult(self, indexName):
        url = f"https://www.nseindia.com/api/corp-info?symbol={indexName}&corpType=financialResult&market=equities"
        response = self.s.get(url, verify=self.ssl_verify)

        if response.status_code == 200:
            return pd.DataFrame(response.json())
        else:
            raise Exception(f"Failed to fetch data: {response.status_code} {response.text}")

    # Shareholding Patterns
    @ensure_session
    def getShareholdingPatterns(self, StockName):
        url = f"https://www.nseindia.com/api/top-corp-info?symbol={StockName}&market=equities"
        response = self.s.get(url, verify=self.ssl_verify)

        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Failed to fetch data: {response.status_code} {response.text}")

    # Quote Information
    @ensure_session
    def getQuoteInformation(self, StockName):
        url = f"https://www.nseindia.com/api/quote-equity?symbol={StockName}"
        response = self.s.get(url, verify=self.ssl_verify)

        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Failed to fetch data: {response.status_code} {response.text}")

    # Financial Results Comparison
    @ensure_session
    def getFinancialResultsComparison(self, StockName):
        url = f"https://www.nseindia.com/api/corp-info?symbol={StockName}&corpType=financialResult&market=equities"
        response = self.s.get(url, verify=self.ssl_verify)

        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Failed to fetch data: {response.status_code} {response.text}")

    # Insider Trading
    @ensure_session
    def getInsiderTrading(self, StockName):
        url = f"https://www.nseindia.com/api/corp-info?symbol={StockName}&corpType=insiderTrading&market=equities"
        response = self.s.get(url, verify=self.ssl_verify)

        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Failed to fetch data: {response.status_code} {response.text}")



In [112]:
my_nse = MyNSE()

Session Generated Successfully


In [113]:
# dfNSEStockList = my_nse.getNSEStockList("NIFTY%20SMALLCAP%20250")
dfNSEStockList = my_nse.getNSEStockList("NIFTY%20TOTAL%20MARKET")
# dfNSEStockList

In [ ]:
# dfFinancialResult = my_nse.getFinancialResult("HCLTECH")
# dfFinancialResult

In [ ]:
# dfShareholdingPatterns =my_nse.getShareholdingPatterns("HCLTECH")
# dfShareholdingPatterns

In [ ]:
# dfgetQuoteInformation = my_nse.getQuoteInformation("HCLTECH")
# dfgetQuoteInformation

In [ ]:
# dfFinancialResultsComparison=my_nse.getFinancialResultsComparison("HCLTECH")
# dfFinancialResultsComparison

In [ ]:
# dfInsiderTrading = my_nse.getInsiderTrading("HCLTECH")
# dfInsiderTrading

**Analysis  using NSE List Base Filter By Jay patel**

In [114]:
dfNSEStockList.iloc[-1]

priority                                                             0
symbol                                                      KALAMANDIR
identifier                                               KALAMANDIREQN
series                                                              EQ
open                                                            188.79
dayHigh                                                          189.5
dayLow                                                           176.9
lastPrice                                                        178.0
previousClose                                                    189.1
change                                                           -11.1
pChange                                                          -5.87
totalTradedVolume                                               838072
totalTradedValue                                           152068164.4
lastUpdateTime                                    05-Jul-2024 15:59:20
yearHi

In [123]:
dfNSEStockList[
  (dfNSEStockList["lastPrice"]<500) &
  (dfNSEStockList["pChange"]>=0) &
  (dfNSEStockList["nearWKH"]<=20) &
  (pd.to_numeric(dfNSEStockList['perChange365d'], errors='coerce')>=50)&
  (pd.to_numeric(dfNSEStockList['perChange30d'], errors='coerce')>=5)
].sort_values(by=['lastPrice'], ascending=True)
# totalTradedVolume

,priority,symbol,identifier,series,open,dayHigh,dayLow,lastPrice,previousClose,change,...,nearWKH,nearWKL,perChange365d,date365dAgo,chart365dPath,date30dAgo,perChange30d,chart30dPath,chartTodayPath,meta
61,0,RPOWER,RPOWEREQN,EQ,28.20,29.38,28.03,29.07,28.11,0.96,...,15.616836,-103.286713,89.74,06-Jul-2023,https://nsearchives.nseindia.com/365d/RPOWER-E...,05-Jun-2024,23.53,https://nsearchives.nseindia.com/30d/RPOWER-EQ...,https://nsearchives.nseindia.com/today/RPOWERE...,"{'symbol': 'RPOWER', 'companyName': 'Reliance ..."
230,0,HCC,HCCEQN,EQ,47.75,49.00,47.55,48.25,47.81,0.44,...,9.644195,-170.459641,133.04,06-Jul-2023,https://nsearchives.nseindia.com/365d/HCC-EQ.svg,05-Jun-2024,37.63,https://nsearchives.nseindia.com/30d/HCC-EQ.svg,https://nsearchives.nseindia.com/today/HCCEQN.svg,"{'symbol': 'HCC', 'companyName': 'Hindustan Co..."
303,0,DHANI,DHANIEQN,EQ,56.10,57.65,55.74,56.22,55.93,0.29,...,5.986622,-88.026756,73.76,06-Jul-2023,https://nsearchives.nseindia.com/365d/DHANI-EQ...,05-Jun-2024,37.94,https://nsearchives.nseindia.com/30d/DHANI-EQ.svg,https://nsearchives.nseindia.com/today/DHANIEQ...,"{'symbol': 'DHANI', 'companyName': 'Dhani Serv..."
384,0,IFCI,IFCIEQN,EQ,61.24,61.81,60.70,60.97,60.87,0.10,...,15.083565,-416.694915,389.24,06-Jul-2023,https://nsearchives.nseindia.com/365d/IFCI-EQ.svg,05-Jun-2024,15.25,https://nsearchives.nseindia.com/30d/IFCI-EQ.svg,https://nsearchives.nseindia.com/today/IFCIEQN...,"{'symbol': 'IFCI', 'companyName': 'IFCI Limite..."
279,0,RTNINDIA,RTNINDIAEQN,EQ,81.15,83.80,80.79,81.65,81.13,0.52,...,13.916711,-108.556833,101.58,06-Jul-2023,https://nsearchives.nseindia.com/365d/RTNINDIA...,05-Jun-2024,18.52,https://nsearchives.nseindia.com/30d/RTNINDIA-...,https://nsearchives.nseindia.com/today/RTNINDI...,"{'symbol': 'RTNINDIA', 'companyName': 'RattanI..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,RVNL,RVNLEQN,EQ,424.95,498.50,424.55,492.15,418.75,73.40,...,1.273821,-320.461341,299.72,06-Jul-2023,https://nsearchives.nseindia.com/365d/RVNL-EQ.svg,05-Jun-2024,38.99,https://nsearchives.nseindia.com/30d/RVNL-EQ.svg,https://nsearchives.nseindia.com/today/RVNLEQN...,"{'symbol': 'RVNL', 'companyName': 'Rail Vikas ..."
41,0,MIDHANI,MIDHANIEQN,EQ,473.00,499.50,469.10,494.00,472.20,21.80,...,9.771689,-72.065482,68.24,06-Jul-2023,https://nsearchives.nseindia.com/365d/MIDHANI-...,05-Jun-2024,30.31,https://nsearchives.nseindia.com/30d/MIDHANI-E...,https://nsearchives.nseindia.com/today/MIDHANI...,"{'symbol': 'MIDHANI', 'companyName': 'Mishra D..."
221,0,KALYANKJIL,KALYANKJILEQN,EQ,496.55,505.00,485.10,495.40,490.55,4.85,...,9.267399,-238.619275,221.94,06-Jul-2023,https://nsearchives.nseindia.com/365d/KALYANKJ...,05-Jun-2024,23.04,https://nsearchives.nseindia.com/30d/KALYANKJI...,https://nsearchives.nseindia.com/today/KALYANK...,"{'symbol': 'KALYANKJIL', 'companyName': 'Kalya..."
179,0,PRICOLLTD,PRICOLLTDEQN,EQ,494.45,498.00,490.55,496.35,490.25,6.10,...,3.621359,-135.236967,119.13,06-Jul-2023,https://nsearchives.nseindia.com/365d/PRICOLLT...,05-Jun-2024,17.38,https://nsearchives.nseindia.com/30d/PRICOLLTD...,https://nsearchives.nseindia.com/today/PRICOLL...,"{'symbol': 'PRICOLLTD', 'companyName': 'Pricol..."
